### Adaptive Fault Detection with VAR Models

##### Import libraries needed

In [16]:
# importing packages and libraries
from pandas import read_csv
import pandas as pd
import numpy as np
import pickle
import os
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

# Ignoe harmless warnings
import warnings
warnings.filterwarnings("ignore")

# Define the plot size default
from pylab import rcParams
rcParams['figure.figsize'] = (12, 5)

### Defining methods or functions

In [17]:
# Plotting multiple series
def plot_multiple_series(actual, pred, attr):
    for i in range(len(attr)):
        title = "Prediction of {}".format(attr[i])
        plt.title(title)
        plt.xlabel("Timestep")
        plt.ylabel("Values")
        plt.plot(actual.iloc[:,i], label="actual")
        plt.plot(pred.iloc[:,i], label="forecast")
        plt.legend()
        plt.show()


# Root mean squared error
def root_mse(x, y):
    if len(x) != len(y):
        return "Error: The two arguments must have the same length"
    mse = np.square(np.subtract(x, y)).mean()
    return np.sqrt(mse)

# Plotting series
def plot_series(series, attr):
    for i in range(len(attr)):
        title = "Plot of "+str(attr[i])
        actual = series.iloc[:,i]
        plt.title(title)
        plt.xlabel("Timestep")
        plt.ylabel(attr[i])
        plt.plot(actual)
        plt.show()

# Normalisation of time series
def normalise_timeseries(data):
    # Calculate the mean and standard deviation for each feature
    means = np.mean(data, axis=0)
    stds = np.std(data, axis=0)
    
    # Normalise each feature using standard deviation
    normalised_data = (data - means) / stds
    return pd.DataFrame(normalised_data)


# Denomalisation of time series
def denormalise_timeseries(data, means, stds):
    denormalised_data = (data * stds) + means
    return pd.DataFrame(denormalised_data)


# Augmented Dickey-Fuller Test
def adf_test(series, title=''):
    '''
    Hypothesis Test for Stationarity
    Pass in a time series and an optional title, return an ADF report
    '''
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(),autolag='AIC')
    labels = ['ADF test statistics','p-value','#lags','#observations'] # use help(adfuller) to understand why these labels are chosen
    
    outcome = pd.Series(result[0:4],index=labels)
    
    for key,val in result[4].items():
        outcome[f'critical value ({key})'] = val
        
    print(outcome.to_string()) # this will not print the line 'dtype:float64'
    
    if result[1] <= 0.05:
        print('Strong evidence against the null hypothesis') # Ho is Data is not stationary, check help(adfuller)
        print('Reject the null hypothesis')
        print('Data is Stationary')
    else:
        print('Weak evidence against the Null hypothesis')
        print('Fail to reject the null hypothesis')
        print('Data has a unit root and is non stationary')


# Loading expert models in a dictionary
def load_expert_models(expert_path):
    files = os.listdir(expert_path)
    pickle_files = [file for file in files if file.endswith('.pkl')]
    models = {}

    for file in pickle_files:
        with open(file, 'rb') as f:
            models[file.split('.')[0]] = pickle.load(f)

    return models


In [18]:
# Assigning variables
# file = 'test_series_reduced.csv'
# file = 'stuckat1_training_reduced_new.csv'
file = 'valueflip_training_reduced_new.csv'
# file = 'stuckat1_training_reduced.csv'
expert_path = 'expert_models'
df_raw = read_csv(file, header=0, index_col=0)
attr = list(pd.read_csv(file).columns.values)[1:]
series = df_raw.iloc[:40000,:]
# plot_series(series, attr)
nobs = 3000
steps = 15
begin = 2000
finish = 2200
normalised_data = normalise_timeseries(df_raw)
testData = normalised_data.copy()
train = testData.iloc[:-nobs]
test = testData.iloc[-nobs:]
# len(train), len(test)
input1 = testData.iloc[begin:finish,:]
# plot_series(input1, attr)
prediction_error = {}

In [19]:
# Threshold prediction error value
threshold = 0

In [20]:
# Load the expert models
my_experts = load_expert_models(expert_path)
# print(my_experts)
print(my_experts['var_ctrl_stuckat0_perm_reduced'])


In [21]:
pred = my_experts['var_ctrl_stuckat0_perm_reduced'].forecast(input1.values, steps=steps)
pred_df = pd.DataFrame(pred, columns=input1.columns)
print(pred_df)

    Tank2OutFlow  Tank2.puddle  Tank3OutFlow  Tank2.level  wt3_valve
0       0.166941     -1.412529     -3.252720     0.040152  -2.849662
1       0.141700     -1.413880     -3.841274     0.590707  -1.091605
2       0.125553     -1.416389     -2.842014     0.845684  -2.906553
3       0.143341     -1.417214     -3.380680     1.324351  -1.242575
4       0.172841     -1.418933     -2.440453     1.545943  -2.931201
5       0.164743     -1.419439     -2.970947     1.959229  -1.345504
6       0.128176     -1.420603     -2.085089     2.153137  -2.904582
7       0.124630     -1.420797     -2.569727     2.501561  -1.359546
8       0.153424     -1.421382     -1.629034     2.657553  -2.977493
9       0.170109     -1.421457     -2.398997     2.964665  -1.487252
10      0.151343     -1.421773     -1.431840     3.113451  -2.891778
11      0.126945     -1.421606     -2.083941     3.371359  -1.536687
12      0.127460     -1.421550     -1.195921     3.499899  -2.744520
13      0.152256     -1.421207    

In [22]:
for model_name, model in my_experts.items():
    predictions = my_experts[model_name].forecast(input1.values, steps=steps)
    predictions_df = pd.DataFrame(predictions, columns=input1.columns)
    expected = testData.iloc[finish:finish+steps,:].reset_index(drop=True)
    RMSE = []
    for feature in attr:
        RMSE.append(root_mse(predictions_df[feature], expected[feature]))

    prediction_error[model_name] = RMSE
    
print(prediction_error)

{'var_ctrl_stuckat0_perm_reduced': [0.004098851929610022, 0.011669309952683815, 2.013310786919584, 2.238071436133283, 1.7539182512583371], 'var_ctrl_stuckat1_perm_reduced': [0.003265618792926763, 0.008634230717338187, 1.1196592083017551, 0.36831143991488235, 1.1272382517515824], 'var_ctrl_valueFlip_perm_reduced': [0.003349691468002341, 0.0005376934063501672, 0.6108575334145131, 0.17428323994596506, 0.8088449316012353], 'var_golden_model_reduced': [0.009505151971532736, 0.002325238684490896, 0.8614289289268174, 0.512059735099321, 0.7280663832150845]}


In [23]:
for key, value in prediction_error.items():
    print(prediction_error[key], np.mean(prediction_error[key]))

[0.004098851929610022, 0.011669309952683815, 2.013310786919584, 2.238071436133283, 1.7539182512583371] 1.2042137272386995
[0.003265618792926763, 0.008634230717338187, 1.1196592083017551, 0.36831143991488235, 1.1272382517515824] 0.525421749895697
[0.003349691468002341, 0.0005376934063501672, 0.6108575334145131, 0.17428323994596506, 0.8088449316012353] 0.3195746179672132
[0.009505151971532736, 0.002325238684490896, 0.8614289289268174, 0.512059735099321, 0.7280663832150845] 0.42267708757944933


In [24]:
type(prediction_error)

dict

In [25]:
def identify_fault(error_dict):
    best_val = 100
    best_expert = ""
    for key, value in error_dict.items():
        if np.mean(error_dict[key]) < best_val:
            best_val = np.mean(error_dict[key])
            best_expert = key
    print(f'The best expert is {best_expert}')

In [26]:
identify_fault(prediction_error)

The best expert is var_ctrl_valueFlip_perm_reduced


In [27]:
print("Temporarily stop execution")
print(stop)

Temporarily stop execution


NameError: name 'stop' is not defined

In [ ]:
test_file = 'test_outputs.csv'
# test_file = 'stuckat1_training_reduced.csv'
# test_file = 'test_series_reduced.csv'
# dff = pd.read_csv(test_file, index_col=0, header=0, parse_dates=True)
dff = pd.read_csv(test_file, index_col=0, header=0)
featt = list(pd.read_csv(test_file, index_col=0, header=0).columns.values)[1:]
plot_series(dff, featt)

In [ ]:
def plot_dataframe(df):
    num_series = len(df.columns)
    fig, axes = plt.subplots(num_series, 1, figsize=(10, 5*num_series), sharex=True)
    for i, col in enumerate(df.columns):
        ax = axes[i] if num_series > 1 else axes
        ax.plot(df.index, df[col])
        ax.set_title(col)
        ax.grid(True)

    plt.tight_layout()
    plt.show()